<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-experiments/chat_hf_local_summarize_graham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y haystack-ai llmx transformers

Found existing installation: haystack-ai 2.0.0b4
Uninstalling haystack-ai-2.0.0b4:
  Successfully uninstalled haystack-ai-2.0.0b4
Found existing installation: transformers 4.37.0.dev0
Uninstalling transformers-4.37.0.dev0:
  Successfully uninstalled transformers-4.37.0.dev0


In [2]:
!pip install -q autoawq accelerate git+https://github.com/huggingface/transformers.git git+https://github.com/deepset-ai/haystack.git@hf_chat_support

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch

from haystack.components.generators.utils import default_streaming_callback
from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.generators.chat import HuggingFaceLocalChatGenerator
from haystack.dataclasses import ChatMessage
from haystack import Pipeline

In [4]:
lcf = LinkContentFetcher(user_agents=["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"])
html_converter = HTMLToDocument(extractor_type="ArticleExtractor")

prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])

In [5]:
llm = HuggingFaceLocalChatGenerator(model="TheBloke/OpenHermes-2.5-Mistral-7B-16k-AWQ",
                                    huggingface_pipeline_kwargs={"device_map": "auto"},
                                    streaming_callback=default_streaming_callback)

In [6]:
pipe = Pipeline()
pipe.add_component("fetcher", lcf)
pipe.add_component("converter", html_converter)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)


pipe.connect("fetcher.streams", "converter.sources")
pipe.connect("converter.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.messages")

In [7]:
template_prefix = """Given the article below: \n
            {% for document in documents %}
                {{ document.content }}
            {% endfor %}
            {{prompt_suffix}}
            """

messages = [ChatMessage.from_user(template_prefix)]


In [11]:
result = pipe.run(data={"urls": ["https://www.paulgraham.com/superlinear.html"],
                        "prompt_source": messages,
                        "template_variables": {"prompt_suffix" : "Summarize the main takeaways and learnings in a few key bullet points"},
                        "generation_kwargs": {"prompt_lookup_num_tokens": 10}})

- Superlinear returns refer to the concept where the rewards for performance are disproportionately higher than the effort put in.
- The author believes that understanding superlinear returns is crucial for ambitious individuals to succeed in today's world.
- Two main causes of superlinear returns are exponential growth and thresholds.
- Exponential growth can lead to thresholds and vice versa.
- Work that compounds directly (doing well in one cycle causes you to do better in the next) and work that compounds through learning are the most promising.
- Ambitious people should take advantage of superlinear returns by doing exceptional work, choosing work they are interested in, and following their curiosity.
- The list of fields with superlinear returns includes sports, politics, art, music, directing, math, starting companies, and investing.
- Superlinear returns imply inequality, with a few big winners outperforming everyone else.
- The territory of superlinear returns can expand, maki